In [41]:
import torch
import torch.nn as nn

class LSTMRecoveryController(nn.Module):
    def __init__(self, input_size, hidden_size, dropout):
        super(LSTMRecoveryController, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2,dropout=dropout, batch_first=True)
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(in_features=hidden_size, out_features=hidden_size // 2)
        self.fc2 = nn.Linear(in_features=hidden_size // 2, out_features=1)

    def forward(self, x):
        out, (last_h_state, last_c_state) = self.lstm(x)
        last_h_state = last_h_state[-1,:,:]
        x = self.fc1(last_h_state)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)*2 - 1
        return x

In [45]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("./data/ae_data.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,time,pos,vel,est_pos,est_vel,det_est_pos,det_est_vel,measured_vel,reference_vel,ctl_signal,attack,attack_pred,residual,cusum_stat
0,0.00,0.000000,16.442814,0.000000,16.442814,0.159524,16.008867,15.461929,17.442814,1.000000,False,False,-0.490442,0.0
1,0.01,0.202908,16.501936,0.159524,16.008867,0.322726,16.688765,17.360445,17.442814,0.103002,False,False,0.704115,0.0
2,0.02,0.435625,16.518542,0.322726,16.688765,0.491437,17.099771,17.521555,17.442814,-0.058896,False,False,0.432567,0.0
3,0.03,0.588291,16.584260,0.491437,17.099771,0.663320,17.303445,17.517581,17.442814,-0.055669,False,False,0.214265,0.0
4,0.04,0.743183,16.617385,0.663320,17.303445,0.834291,16.865072,16.313931,17.442814,1.000000,False,False,-0.494729,0.0


In [11]:
def preprocess_data(df: pd.DataFrame):

    df.drop(columns=['time', 'pos', 'vel', 'est_pos', 'est_vel', 'attack', 'attack_pred', 'cusum_stat'], inplace=True)
    df['experiment'] = df.index // 5000
    scaler = MinMaxScaler(feature_range=(-1,1))
    df[['det_est_pos', 'det_est_vel', 'measured_vel', 'reference_vel', 'residual']] = scaler.fit_transform(
        df[['det_est_pos', 'det_est_vel', 'measured_vel', 'reference_vel', 'residual']])
    return df

In [31]:
from torch.utils.data import Dataset
import pandas as pd

class RecoverySequenceDataset(Dataset):
    def __init__(self, df: pd.DataFrame, seq_len):
        super().__init__()
        self.seq_len = seq_len
        self.sequences = []
        self.labels = []

        for _, group in df.groupby("experiment"):
            group = group.drop(columns=['experiment'])
            seqs, labels = self.create_sequences(group)
            self.sequences.extend(seqs)
            self.labels.extend(labels)

    def create_sequences(self, group):
        sequences = []
        labels = []
        for i in range(len(group) - self.seq_len+1):
            seq = group.drop(columns=['ctl_signal']).iloc[i:i+self.seq_len].values
            label = group.ctl_signal.iloc[i+self.seq_len-1]
            sequences.append(torch.tensor(seq, dtype=torch.float32))
            labels.append(torch.tensor(label, dtype=torch.float32))
        return sequences, labels

    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, index):
        return self.sequences[index], self.labels[index]

In [43]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
def train_epoch(model: nn.Module, dataloader: DataLoader, criterion, optimizer, writer: SummaryWriter, epoch: int):
    model.train()
    for i, (x_batch, y_batch) in enumerate(dataloader):
        
        optimizer.zero_grad()

        model_out = model(x_batch)
        loss = criterion(model_out, y_batch.unsqueeze(1))
        loss.backward()
        writer.add_scalar("loss/train", loss.item(), global_step=epoch*len(dataloader)+i)

        optimizer.step()

    return model

def eval_model(model: nn.Module, dataloader: DataLoader, criterion, writer: SummaryWriter, epoch: int):
    model.eval()
    total_loss = 0.0
    for i, (x_batch, y_batch) in enumerate(dataloader):
        
        with torch.no_grad():
            model_out = model(x_batch)
            loss = criterion(model_out, y_batch)
            total_loss += loss.item()
    
    total_loss = total_loss / len(dataloader)
    writer.add_scalar("loss/validation", total_loss, global_step=epoch)
    return total_loss

In [ ]:
df = pd.read_csv("./data/ae_data.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = preprocess_data(df)
ds = RecoverySequenceDataset(df, seq_len=4)

In [34]:
df = pd.read_csv('data/ae_data.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)
df = preprocess_data(df)

train_ds = RecoverySequenceDataset(df[~df.experiment.isin([45, 46, 47, 48, 49])], seq_len=50)
val_ds = RecoverySequenceDataset(df[df.experiment.isin([45, 46])], seq_len=50)
# test_ds = SequenceDataset(df[df.experiment.isin([47, 48, 49])], seq_len=50)

In [44]:
from tqdm import tqdm

writer = SummaryWriter(log_dir="./tensorboard_logs")

dataloader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=4, shuffle=True)
# Hyperparams
lr = 0.001

# Training params
NUM_EPOCHS = 1

# Model, Optimizer, Loss function
model = LSTMRecoveryController(input_size=5, hidden_size=8, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr)
criterion = torch.nn.MSELoss()

best_loss = float('inf')
for epoch in tqdm(range(NUM_EPOCHS), desc="Epoch"):
    model = train_epoch(model, dataloader, criterion, optimizer, writer, epoch)
    val_loss = eval_model(model, val_dataloader, criterion, writer, epoch)

    if val_loss < best_loss:
        torch.save(model, f"./models/recoverycontroller_ep{epoch}.pt")
        print(f"Best Validation Loss: {val_loss}")
        best_loss = val_loss

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]/home/axel/miniconda3/envs/cats/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/axel/miniconda3/envs/cats/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch: 100%|██████████| 1/1 [04:00<00:00, 240.08s/it]

Best Validation Loss: 0.7700471053460781


In [47]:
model = torch.load("./models/recoverycontroller_ep19.pt", map_location=torch.device('cpu'))

/tmp/ipykernel_39362/1377846169.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("./models/recoverycontroller_ep19.pt", map_location=torch.device('cpu'

In [87]:
torch.save(model.state_dict(), "./models/controller_weights.pth")

In [90]:
model = LSTMRecoveryController(input_size=5, hidden_size=8, dropout=0.2)
model.load_state_dict(torch.load("./models/controller_weights.pth", weights_only=True))

<All keys matched successfully>

In [75]:
datapoint = torch.rand(size=(1,5), requires_grad=False)
datapoint = datapoint.unsqueeze(0)
datapoint.size()

torch.Size([1, 1, 5])

In [76]:
model(datapoint)

tensor([[0.5898]], grad_fn=<SubBackward0>)

In [86]:
pred = model(datapoint)
pred[0][0].detach().numpy() 

array(0.58984745, dtype=float32)